# <span style="font-family: 'Computer Modern'; font-size: 42pt; font-weight: bold;">Quantum Convolutional Neural Network (QCNN) Using *PennyLane*</span>

***

In [1]:
### IMPORTS / DEPENDENCIES:

# PennyLane:
import pennylane as qml
from pennylane import numpy as np
import torch

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches # Quantum Circuit Drawings
# mpl.rcParams.update(mpl.rcParamsDefault)
# from tqdm import tqdm
# import csv
 
# import math
# import random

from scipy.linalg import expm # Unitary-Related Operations

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 26pt;">THE MNIST DATASET</span>

<img src="qcnn-figures/mnist_plot.png" alt="MNIST Dataset Sample Images" style="display: block; margin-left: auto; margin-right: auto; width: 80%;">

<p style="text-align: center; font-family: 'Computer Modern', serif;">
    Sample of the handwritten digital pixelations from the MNIST dataset, which are used for training and testing the QCNN model.<br>
    <em>Image source: <a href="https://corochann.com/mnist-dataset-introduction-532/">https://corochann.com/mnist-dataset-introduction-532/</a></em>
</p>

<span style="font-family: 'Computer Modern'; font-size: 16pt; font-weight: bold;">Loading the MNIST Dataset:</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">For our QCNN, we load the MNIST dataset using TorchVision, which allows us to process the data with quantum features and pass it into our neural network. We define the path for the MNIST data directory below, and use TorchVision to load in the MNIST dataset (Note:  the exact "path name" that you choose can be arbitrary and/or at your discretion, as our dataloaders will be able to handle the data loading under most root name cases). We then initialize the batch sizes for the MNIST training and testing data sets. In this model, we set the batch size for the training data at 350, and at 250 for the testing data.</span>

In [2]:
# Import relevant class(es) for MNIST DATA LOADING AND PROCESSING before passing data to QC:
from lppc_qcnn.qc_data import DataLPPC as lppc_data

In [3]:
### READING AND LOADING DATA: 
### REQUIRED CLASSES: DataLPPC


# Set directory for data:
data_path = './DATA'

# Set batch sizes for training and testing data:
batch_train_qcnn = 350
batch_test_qcnn = 250

# Note: Selections of batch_train=350 and batch_test=250 were chosen for our own preferred sample size, and is
# also up to your own discretion.
train_images, train_labels, test_images, test_labels = lppc_data.load_mnist_torch(batch_train=batch_train_qcnn,
                                                                    batch_test=batch_test_qcnn, root=data_path)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"test_labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")

train_images shape: torch.Size([350, 1, 28, 28]), dtype: torch.float32
test_images shape: torch.Size([250, 1, 28, 28]), dtype: torch.float32
train_labels shape: torch.Size([350]), dtype: torch.int64
test_labels shape: torch.Size([250]), dtype: torch.int64


<span style="font-family: 'Computer Modern'; font-size: 16pt; font-weight: bold;">MNIST DATA TRANSFORMATIONS:</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">We initialize the reduction sizes for the MNIST training and testing data sets. In this model, we set the reduction size for the training data at 500, and at 100 for the testing data. We then reduce the number of data points in the training and testing datasets as necessary (Note: it is important to ensure that at least one of the specified reduction values for "n_train" and "n_test" is smaller than its  corresponding batch size values used during the loading step for the MNIST data, or else no reduction stage is necessary in the steps for the model).</span>

In [4]:
### REDUCING THE IMPORTED MNIST DATA
### REQUIRED CLASSES: DataLPPC

# Reduction sizes:
n_train_qcnn = 500
n_test_qcnn = 100

# Reduce datasets as needed:
if n_train_qcnn < batch_train_qcnn or n_test_qcnn < batch_test_qcnn:
    train_images, train_labels, test_images, test_labels = lppc_data.mnist_reduce(train_images, train_labels,
                                        test_images, test_labels, n_train=n_train_qcnn, n_test=n_test_qcnn)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"test_labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")

train_images shape: torch.Size([76, 1, 28, 28]), dtype: torch.float64
test_images shape: torch.Size([70, 1, 28, 28]), dtype: torch.float64
train_labels shape: torch.Size([76]), dtype: torch.float64
test_labels shape: torch.Size([70]), dtype: torch.float64


In [5]:
### FLATTENING THE IMPORTED MNIST DATA
### REQUIRED CLASSES: DataLPPC

train_images, test_images = lppc_data.mnist_flatten(train_images, test_images)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"test_images shape: {test_images.shape}, dtype: {test_images.dtype}")

train_images shape: torch.Size([76, 784]), dtype: torch.float64
test_images shape: torch.Size([70, 784]), dtype: torch.float64


In [6]:
### PADDING THE FLATTENED DATASETS
### REQUIRED CLASSES: DataLPPC

x_train, y_train, x_test, y_test = lppc_data.mnist_padding(train_images, train_labels,
                                                           test_images, test_labels)

# Print relevant shapes and types of your training and testing data to check progress:
print(f"x_train shape: {x_train.shape}, dtype: {x_train.dtype}")
print(f"x_test shape: {x_test.shape}, dtype: {x_test.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

x_train shape: (76, 1024), dtype: float64
x_test shape: (70, 1024), dtype: float64
y_train shape: (76,), dtype: float64
y_test shape: (70,), dtype: float64


***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">QCNN MODEL</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">*Discuss QCNN model structure and layering here.*</span>

In [7]:
# Import relevant classclass(es) for QUANTUM CIRCUIT (before passing weights to the QC):
from lppc_qcnn.gellmann_ops import ParamOps as param_ops # PARAMETER OPERATIONS HELPER CLASS

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Trainable Parameters_:</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">We prepare the trainable parameters (weights) for the QCNN by properly transforming their type and shape. We ensure that the weights are Torch tensors of a relevant datatype, and also ensure there are enough weights to train on to be able to pass to the QC, and subsequently through our stochastic gradient descent training loop.</span>

In [8]:
### INITIALIZING QUBIT PARAMETERS AND WEIGHTS
### REQUIRED CLASSES: ParamOps

n_qubits = 2  # Number of qubits
# n_qubits = 10
active_qubits = 2 # Number of active qubits (same as n_qubits, tracks QC operations)
# active_qubits = 10

# TODO
qcnn_weights0 = np.random.uniform(0, np.pi, size=(n_qubits,1,3))

# Prepare weights and transform them by passing to 'param_prep_lppc':
qcnn_weights = param_ops.broadcast_weights_lppc(param_ops, qcnn_weights0)

# Print relevant attributes of 'qcnn_weights':
print(f"Size of 'qcnn_weights': {qcnn_weights.size}")
print(f"Shape of 'qcnn_weights': {qcnn_weights.shape}")
print(f"Length of 'qcnn_weights': {len(qcnn_weights)}")
print(f"Type of 'qcnn_weights': {type(qcnn_weights)}")
print(f"Type of an element of 'qcnn_weights': {type(qcnn_weights[0])}")
print(f"Data type of elements in 'qcnn_weights': {qcnn_weights.dtype}")

Size of 'qcnn_weights': 6
Shape of 'qcnn_weights': (6,)
Length of 'qcnn_weights': 6
Type of 'qcnn_weights': <class 'pennylane.numpy.tensor.tensor'>
Type of an element of 'qcnn_weights': <class 'pennylane.numpy.tensor.tensor'>
Data type of elements in 'qcnn_weights': float64


<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">CIRCUIT DRAWING</span>

In [9]:
# Import relevant class(es) for CIRCUIT CONSTRUCTION ANF DRAWING prior to visualizations:
from lppc_qcnn.qcircuit import QCircuitLPPC as qc_circ # QUANTUM CIRCUIT CLASS

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">TRAINING / OPTIMIZATION </span>

In [10]:
# Import relevant class(es) for TRAINING AND OPTIMIZATION-RELATED PROCESSES prior to training weights:
from lppc_qcnn.qcircuit import OptStepLPPC as opt_lppc # OPTIMIZATION AND COST CLASS

# OPTIMIZER CHECK:
# opt_alpha = qml.GradientDescentOptimizer()
# opt_alpha.step_and_cost?

Signature: opt_alpha.step_and_cost(objective_fn, *args, grad_fn=None, **kwargs)
Docstring:
Update trainable arguments with one step of the optimizer and return the corresponding
objective function value prior to the step.

Args:
    objective_fn (function): the objective function for optimization
    *args : variable length argument list for objective function
    grad_fn (function): optional gradient function of the
        objective function with respect to the variables ``*args``.
        If ``None``, the gradient function is computed automatically.
        Must return a ``tuple[array]`` with the same number of elements as ``*args``.
        Each array of the tuple should have the same shape as the corresponding argument.
    **kwargs : variable length of keyword arguments for the objective function

Returns:
    tuple[list [array], float]: the new variable values :math:`x^{(t+1)}` and the objective
    function output prior to the step.
    If single arg is provided, list [array] i

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Training Model_:</span>

In [11]:
### OPTIMIZATION AND TRAINING
### REQUIRED CLASSES: OptStepLPPC, QCircuitLPPC, GellMannOps


n_qubits = 2  # Number of qubits
active_qubits = 2 # Number of active qubits (same as n_qubits, tracks QC operations)

# Initialize the selected optimizer (Note: in this model, the Stochastic Gradient Descent (SGD) Optimizer was 
# determined to be the most suitable, although the choice of optimizer is additionally up to your own discretion.)

# Set value to an integer 1-6 based on desired optimizer selection from 'opt' (Note: For this model, "1" corresponds 
# to the Stochastic Gradient Descent (SGD) Optimizer ('opt_num'=1). You can use qc_opt_print() to see all available
# optimizers to choose from.
# opt_num_lppc = 1 # TAKE AS PARAMETER

# List of all available / acceptable optimizers for QCNN model (Note: this optimizer list does not include
# all of the available optimizer selections in PennyLane, it only includes 6 that were selected based on efficiency
# and relevance to our model and data):
# Optimizer List:
#    1: qml.GradientDescentOptimizer, # <- PRIMARY OPTIMIZER
#    2: qml.AdamOptimizer,
#    3: qml.RMSPropOptimizer,
#    4: qml.MomentumOptimizer,
#    5: qml.NesterovMomentumOptimizer,
#    6: qml.AdagradOptimizer

# Select Stochastic Gradient Descent (SGD) Optimizer:
# opt = opt_lppc.qcnn_opt_select(opt_num_lppc)
opt = qml.GradientDescentOptimizer()

# Initialize optimization Parameters:
learning_rate = 0.1
batch_size = 10
max_iter = 100
conv_tol = 1e-06

# Initialize Training History Parameters:
num_steps = 10
loss_history = []
# Instantiate below for access the model's loss history during training:
# hist_lppc = True

# Training Loop:
for step in range(num_steps):
    qcnn_weights, loss = opt_lppc.stoch_grad_lppc(opt_lppc, opt, opt_lppc.mse_cost, qcnn_weights, x_train, y_train,
                                                learning_rate, batch_size, max_iter, conv_tol)
    
    loss_history.append(loss)  # Accumulate loss

    # Print step and cost:
    print(f"Step {step}: cost = {loss}")

# Evaluate Optimization Accuracy on testing dataset:
predictions = np.array([qc_circ.qcircuit_lppc(qc_circ, qcnn_weights, xi) for xi in x_test])

ValueError: Features must be of length 4; got length 1024. Use the 'pad_with' argument for automated padding.

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Accuracy_:</span>

In [ ]:
### PREDICTIONS AND ACCURACY
### REQUIRED CLASSES: OptStepLPPC

# Calculate and determine accuracy of the current QCNN model:
accuracy = opt_lppc.accuracy_lppc(opt_lppc, predictions, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

***

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 20pt;">_APPENDIX_</span>

In [ ]:
# TODO

***

<p style="font-family: 'Computer Modern'; font-size: 10pt; font-weight: bold; text-align: center;">
    © The Laboratory for Particle Physics and Cosmology (LPPC) at Harvard University, Cambridge, MA<br>
    © Sean Chisholm<br>
    © Pavel Zhelnin
</p>

***